In [83]:
import os
def txt_finder(atlas_txt_dir, colour, avg):
    '''
    Finds the appropiate cleaned+averaged txt file for lightcurves
    '''
    txt_files = []
    for root, dirs, files in os.walk(atlas_txt_dir):
        for file in files:
            if file.endswith(f'{colour}.{avg}.00days.lc.txt'):
                txt_files.append(os.path.join(root, file))

    if txt_files:
        #logging.info(f'{txt_files}')
        return txt_files
    else:
        #logging.info('no light curve data processed')
        return

In [84]:
atlas_output_dir = r'/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output'
c_txt = txt_finder(atlas_txt_dir=atlas_output_dir, colour='c',avg=4)
o_txt = txt_finder(atlas_txt_dir=atlas_output_dir, colour='o', avg=4)

In [85]:
c_txt

['/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246_galaxy0\\neutrino_139939_8268246_galaxy0.c.4.00days.lc.txt',
 '/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246_galaxy1\\neutrino_139939_8268246_galaxy1.c.4.00days.lc.txt',
 '/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246_galaxy10\\neutrino_139939_8268246_galaxy10.c.4.00days.lc.txt',
 '/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246_galaxy11\\neutrino_139939_8268246_galaxy11.c.4.00days.lc.txt',
 '/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246_galaxy12\\neutrino_139939_8268246_galaxy12.c.4.00days.lc.txt',
 '/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/atlas_query/output\\neutrino_139939\\neutrino_139939_8268246

In [75]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

def plot_averaged_SN(
    avg_sn,
    name,
):
    fig, ax1 = plt.subplots(1, constrained_layout=True)
    fig.set_figwidth(7)
    fig.set_figheight(4)

    title = f"Cleaned & averaged SN"
    ax1.set_title(title)

    ax1.minorticks_on()
    ax1.tick_params(direction="in", which="both")
    ax1.set_ylabel(r"Flux ($\mu$Jy)")
    ax1.set_xlabel("MJD")
    ax1.axhline(linewidth=1, color="k")


    plt.errorbar(
        avg_sn["MJD"],
        avg_sn["uJy"],
        yerr=avg_sn['duJy'],
        fmt="none",
        elinewidth=1,
        capsize=1.2,
        alpha=0.5,
        zorder=10,
    )
    plt.scatter(
        avg_sn["MJD"],
        avg_sn["uJy"],
        marker="o",
        alpha=0.5,
        zorder=10,
        label=name,
    )

    ax1.legend(loc="upper right", facecolor="white", framealpha=1.0).set_zorder(100)

    return fig

In [76]:
def plotter(txt_files, output_dir):
    '''
    Compiles all plots and outputs to final_output dir
    '''
    # Customize the plot
    plt.figure(figsize=(10, 6))
    plt.title("Flux vs. Time (MJD)")
    plt.xlabel("Time (MJD)")
    plt.ylabel("Flux (uJy)")
    plt.grid(True)

    for file in txt_files:
        df = pd.read_csv(file, delim_whitespace=True)
        filtered_df = df[(df['uJy'].notna()) & (df['MJD'].notna())]
        dir_name = os.path.basename(os.path.dirname(file))
        galaxy_name = dir_name.split('_')[-1] if 'galaxy' in dir_name else None
        plot_averaged_SN(filtered_df, galaxy_name)
        
        #plt.errorbar(filtered_df['MJD'], filtered_df['uJy'], marker='o', label='galaxy0')
        #plt.legend(loc="upper right")
        #plt.savefig(os.path.join(output_dir, 'compiled_plot.png'))


In [82]:
final_output_dir = r'/users/jhzhe/Cloned_Repos/MPhys-Project/output_data/final_output'


In [118]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

def plot_averaged_SN(avg_sn, name, ax):
    """
    Plot individual galaxy data on the provided axis.

    Args:
        avg_sn (DataFrame): The DataFrame containing averaged SN data.
        name (str): Label for the plot (e.g., galaxy name).
        ax (Axes): The matplotlib axis to plot on.
    """
    ax.errorbar(
        avg_sn["MJD"],
        avg_sn["uJy"],
        yerr=avg_sn['duJy'],
        fmt="none",
        elinewidth=1,
        capsize=1.2,
        alpha=0.5,
        zorder=10,
    )
    ax.scatter(
        avg_sn["MJD"],
        avg_sn["uJy"],
        marker="o",
        alpha=0.5,
        zorder=10,
        label=name,
    )


def plotter(txt_files, output_dir, colour, flag):
    """
    Compiles all plots and outputs them to the output directory.
    
    Args:
        txt_files (list): List of file paths for the light curve data.
        output_dir (str): Directory to save the compiled plots.
    """
    grouped_files = {}
    for file in txt_files:
        # Extract the neutrino ID (e.g., neutrino_139939)
        path_parts = file.replace("\\", "/").split("/")
        neutrino_id = next(part for part in path_parts if part.startswith("neutrino") and "_galaxy" not in part)
        if neutrino_id not in grouped_files:
            grouped_files[neutrino_id] = []
        grouped_files[neutrino_id].append(file)
    # create directory for each id
    id_paths = {}
    for id in grouped_files.keys():
        id_path = os.path.join(final_output_dir, f'{id}')
        id_paths[id] = id_path
        os.makedirs(id_path, exist_ok=True)
    
    if flag:
        # Plot each group of files
        for neutrino_id, files in grouped_files.items():
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.set_title(f"Flux vs. Time for {neutrino_id}")
            ax.set_xlabel("Time (MJD)")
            ax.set_ylabel("Flux (uJy)")
            ax.grid(True)
            for file in files:
                df = pd.read_csv(file, delim_whitespace=True)
                filtered_df = df[(df['uJy'].notna()) & (df['MJD'].notna())]

                # Extract the galaxy name
                dir_name = os.path.basename(os.path.dirname(file))
                galaxy_name = dir_name.split('_')[-1] if 'galaxy' in dir_name else None

                # Plot the data for this galaxy
                plot_averaged_SN(filtered_df, galaxy_name, ax)
            
            ax.legend(loc="upper right", facecolor="white", framealpha=1.0)
            
            # Save the plot
            output_file = os.path.join(id_paths[neutrino_id], f"{neutrino_id}_compiled_plot_{colour}.png")
            plt.savefig(output_file)
            plt.close(fig)

    if not flag:
        # Plot each group of files
        for neutrino_id, files in grouped_files.items():
            output_pdf_path = os.path.join(id_paths[neutrino_id], f'{neutrino_id}_all_{colour}.pdf')
            with PdfPages(output_pdf_path) as pdf:
                files = sorted(files, key=lambda x: int(x.split("galaxy")[-1].split('.')[0]))
                for file in files:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    ax.set_title(f"Flux vs. Time for {neutrino_id}")
                    ax.set_xlabel("Time (MJD)")
                    ax.set_ylabel("Flux (uJy)")
                    ax.grid(True)
                    df = pd.read_csv(file, sep='\s+')
                    filtered_df = df[(df['uJy'].notna()) & (df['MJD'].notna())]

                    # Extract the galaxy name
                    dir_name = os.path.basename(os.path.dirname(file))
                    galaxy_name = dir_name.split('_')[-1] if 'galaxy' in dir_name else None
                    # Plot the data for this galaxy
                    plot_averaged_SN(filtered_df, galaxy_name, ax)
                
                    ax.legend(loc="upper right", facecolor="white", framealpha=1.0)
                    
                    # Save the plot
                    pdf.savefig(fig)
                    plt.close(fig)


# Example usage
plotter(o_txt, final_output_dir, colour='o', flag =None)

<>:96: SyntaxWarning: invalid escape sequence '\s'
<>:96: SyntaxWarning: invalid escape sequence '\s'
C:\Users\jhzhe\AppData\Local\Temp\ipykernel_21984\1006448890.py:96: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep='\s+')


In [94]:
alert_csv.shape

(135, 9)

In [ ]:
alert_coords = alert_csv[['RunNum_EventNum', 'Date', 'RA [deg]', 'Dec [deg]', 'Error90 [arcmin]']]
event_list = [x for x in alert_coords['RunNum_EventNum']]
ra_list = [float(x) for x in alert_coords['RA [deg]']]
dec_list = [float(x) for x in alert_coords['Dec [deg]']]
err_list = [round(float(x)/60, 4) for x in alert_coords['Error90 [arcmin]']]

In [98]:
for (event, ra, dec, err) in zip(event_list, ra_list, dec_list, err_list):
    print(event,ra,dec,err)

140078_30891383 196.1699 8.57 0.6998
139977_2910365 112.329 -33.5285 0.8705
139939_8268246 58.3599 8.3499 0.685
139912_46959751 180.6599 18.92 0.5898
139735_13003768 55.77 31.8299 4.2382
139685_1723093 60.95 49.31 2.95
139674_30853199 354.99 22.67 1.1898
139315_50057906 327.0799 3.06 1.6148
139303_27647445 73.17 1.6399 2.1748
139279_10803235 102.4399 6.3499 0.7048
139204_39158985 25.3999 7.78 4.2998
139205_9784024 89.2099 0.93 1.4025
139204_39158985 25.3999 7.78 4.2998
139100_34742349 239.6299 39.9399 15.38
138938_62272876 348.3999 10.2799 0.895
139071_47725621 72.25 15.7899 1.1398
138890_34080563 357.54 4.2599 1.3
138821_46175426 72.6899 11.4199 1.1898
138821_46175426 72.6899 11.4199 0.2898
138669_4054005 311.48 10.2799 1.2075
138632_31747601 139.0399 0.3699 1.4975
138599_39138591 177.53 53.6199 2.8075
138515_8773328 105.67 47.85 2.4598
138487_60138479 267.16 46.96 2.97
138449_20481611 297.16 1.34 2.35
138415_56188508 143.7899 25.0399 1.09
138354_45413430 163.83 31.8299 2.14
138283_14